## Data label poisoning

In [1]:
import torch
from PIL import Image
import torchvision
import torchvision.transforms as transforms
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')

Using cuda for inference


In [2]:
!pip install torchviz
!pip install torchsummary
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4131 sha256=4dc6cdcca40079e105871e3c5379ec32fc02294b05ad53db919df451cc7af063
  Stored in directory: /root/.cache/pip/wheels/4c/97/88/a02973217949e0db0c9f4346d154085f4725f99c4f15a87094
Successfully built torchviz
  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=31fae1c4a523faf0d158ed9286c0ba83b81ef032858beaaef5ae6621ba88d181
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [21]:
model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_widese_b4', pretrained=False)
model.eval().to(device)
import torch.optim as optim
import torch.nn as nn
num_epochs = 100
learning_rate = 0.0005
weight_decay = 0.0001
momentum = 0.01
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay, momentum=momentum)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20, 40])
criterion = nn.CrossEntropyLoss()
checkpoint = torch.load("/kaggle/input/ef-net/ef_net_78.pth")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

epoch = checkpoint['epoch']
loss = checkpoint['loss']
accuracy = checkpoint['accuracy']
average_loss_test = checkpoint['test_accuracy']
average_loss = checkpoint['test_loss']
learning_rate = checkpoint['learning_rate']

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

import torchvision.datasets as datasets
normalize= transforms.Normalize(mean=[0.49186882, 0.48265398, 0.44717732], std=[0.24697122, 0.24338895, 0.2615926 ])
transform = transforms.Compose([
        transforms.Resize(224),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.05),
        # transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.49186882, 0.48265398, 0.44717732],std=[0.24697122, 0.24338895, 0.2615926 ])
    ])

batch_size = 64
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(
        datasets.CIFAR10(root='./data', train=True, transform=transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(32, 4),
            transforms.ToTensor(),
            normalize,
        ]), download=True),
        batch_size=batch_size, shuffle=True, pin_memory=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(
        datasets.CIFAR10(root='./data', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ]), download=True),
        batch_size=batch_size, shuffle=True, pin_memory=True)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


Accuracy before label poisoning

In [22]:
import numpy as np

model.eval().to(device)
correct_predictions = np.zeros(len(classes))
total_samples = np.zeros(len(classes))

with torch.no_grad():
    for (inputs, targets) in testloader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)

        for label, prediction in zip(targets, predicted):
            if label == prediction:
                correct_predictions[label] += 1
            total_samples[label] += 1

for i, classname in enumerate(classes):
    accuracy = 100 * correct_predictions[i] / total_samples[i]
    print(f'Accuracy on test data {classname:5s} : {accuracy:.2f}%')

Accuracy on test data plane : 84.50%
Accuracy on test data car   : 90.90%
Accuracy on test data bird  : 74.20%
Accuracy on test data cat   : 68.50%
Accuracy on test data deer  : 81.10%
Accuracy on test data dog   : 70.90%
Accuracy on test data frog  : 87.40%
Accuracy on test data horse : 85.90%
Accuracy on test data ship  : 89.80%
Accuracy on test data truck : 89.40%


In [15]:
import torchvision.datasets as datasets

train_labels = np.array(trainset.targets)

target_class = 3
percentage_to_change = 50
#new_label = 5

all_labels = set(range(10))
exclude_label = {target_class}
possible_new_labels = list(all_labels - exclude_label)

num_to_change = int((percentage_to_change / 100) * np.sum(train_labels == target_class))
indices_to_change = np.random.choice(np.where(train_labels == target_class)[0], num_to_change, replace=False)


print("Index\tOriginal Label\tNew Label")
for index in indices_to_change:
    new_label = np.random.choice(possible_new_labels)
    print(f"{index}\t{train_labels[index]}\t{new_label}")
    train_labels[index] = new_label

train_labels[indices_to_change] = new_label
trainset.targets = list(train_labels)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=False)

Index	Original Label	New Label
38829	3	4
46215	3	0
10143	3	8
37697	3	2
18798	3	6
22577	3	6
29247	3	7
4855	3	1
40668	3	4
14574	3	9
4288	3	8
44039	3	6
43257	3	9
7789	3	0
25885	3	4
33955	3	8
20390	3	0
16013	3	6
45667	3	2
41428	3	6
18954	3	2
21287	3	6
44124	3	9
39644	3	7
19496	3	4
43773	3	1
13082	3	9
15266	3	6
39761	3	0
1965	3	5
5445	3	5
47982	3	5
41705	3	5
35549	3	6
3429	3	0
44197	3	9
2279	3	4
1608	3	9
8467	3	6
7492	3	2
5450	3	7
36664	3	2
34914	3	7
27786	3	8
42882	3	2
49054	3	0
10182	3	1
35102	3	5
6724	3	5
37425	3	0
12475	3	8
30389	3	5
14651	3	0
20569	3	1
1845	3	4
17698	3	6
7872	3	9
15300	3	7
17909	3	7
17660	3	1
15974	3	6
20410	3	8
9111	3	2
31610	3	2
31103	3	8
31254	3	5
38825	3	4
46149	3	6
22456	3	6
29011	3	5
35994	3	1
11480	3	4
4628	3	8
32524	3	9
31567	3	1
43551	3	4
22919	3	9
26525	3	6
49009	3	0
44976	3	0
12546	3	0
21250	3	1
4146	3	1
18960	3	7
42501	3	1
34835	3	7
1737	3	1
21351	3	6
27467	3	1
17455	3	2
30192	3	4
37765	3	2
7802	3	1
23094	3	2
18771	3	7
11026	3	1
28447	3	9
23475	3	6
37514	3	

In [24]:
####TEST ONLY
test_labels = np.array(testset.targets)

# Assuming you want to poison the same target class with the same percentage
num_to_change_test = int((percentage_to_change / 100) * np.sum(test_labels == target_class))
indices_to_change_test = np.random.choice(np.where(test_labels == target_class)[0], num_to_change_test, replace=False)

print("Index\tOriginal Label\tNew Label")
for index in indices_to_change_test:
    new_label = np.random.choice(possible_new_labels)
    print(f"{index}\t{test_labels[index]}\t{new_label}")
    test_labels[index] = new_label

testset.targets = list(test_labels)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)


Index	Original Label	New Label
2752	3	8
2747	3	8
9994	3	8
6647	3	4
7313	3	0
3463	3	0
9422	3	5
8754	3	0
3456	3	4
8966	3	6
4147	3	1
3539	3	2
320	3	8
1787	3	6
3390	3	1
2157	3	9
4342	3	0
3049	3	9
6744	3	6
790	3	9
9246	3	1
5413	3	7
9141	3	7
1219	3	7
3259	3	6
9198	3	0
7813	3	7
9201	3	9
9087	3	8
8484	3	9
3065	3	8
1597	3	5
986	3	7
957	3	9
294	3	2
7496	3	0
5656	3	4
2815	3	7
9109	3	8
3353	3	7
8338	3	7
7644	3	8
3743	3	0
637	3	9
1813	3	5
8665	3	2
7598	3	7
6825	3	1
8331	3	7
7002	3	1
2144	3	0
2529	3	7
5198	3	6
9498	3	2
2049	3	1
4785	3	7
8778	3	8
1074	3	6
695	3	9
1775	3	6
1936	3	5
7228	3	8
2660	3	9
6340	3	9
911	3	6
4938	3	8
9338	3	4
46	3	2
5597	3	7
7310	3	4
8196	3	5
8299	3	6
2922	3	4
5424	3	2
4862	3	6
9748	3	7
7255	3	0
9292	3	8
6774	3	1
3479	3	2
1053	3	2
2720	3	1
6514	3	9
2173	3	4
9926	3	5
2422	3	1
9578	3	8
6406	3	6
3067	3	7
1030	3	8
8114	3	2
1871	3	0
2081	3	2
9404	3	9
1704	3	2
8181	3	8
9454	3	8
5361	3	9
9043	3	1
4718	3	9
8224	3	5
9308	3	7
6174	3	8
6976	3	6
5490	3	6
5272	3	1
7268	3	2
7211	3	0
1957	3	

In [25]:
import numpy as np

model.eval().to(device)
correct_predictions = np.zeros(len(classes))
total_samples = np.zeros(len(classes))

with torch.no_grad():
    for (inputs, targets) in testloader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)

        for label, prediction in zip(targets, predicted):
            if label == prediction:
                correct_predictions[label] += 1
            total_samples[label] += 1

for i, classname in enumerate(classes):
    accuracy = 100 * correct_predictions[i] / total_samples[i]
    print(f'Accuracy on test data {classname:5s} : {accuracy:.2f}%')

Accuracy on test data plane : 0.38%
Accuracy on test data car   : 0.00%
Accuracy on test data bird  : 30.21%
Accuracy on test data cat   : 57.80%
Accuracy on test data deer  : 0.29%
Accuracy on test data dog   : 10.82%
Accuracy on test data frog  : 0.00%
Accuracy on test data horse : 0.00%
Accuracy on test data ship  : 0.00%
Accuracy on test data truck : 0.66%


Accuracy after data poisoning